In [9]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, Dense, GlobalAveragePooling1D
from tensorflow.keras.preprocessing.sequence import pad_sequences
import numpy as np
from math import sin, cos, tan, radians
import pyttsx3 

In [10]:
tf.config.list_physical_devices()

[PhysicalDevice(name='/physical_device:CPU:0', device_type='CPU'),
 PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]

In [ ]:
data = np.load("data.npz", allow_pickle=True)
features = data["features"]
labels = data["labels"]
word_dict = data["word_dict"]

In [5]:
rev_word_dict = dict([(k, v) for v , k in word_dict.items()])
def get_sent(lst):
    return " ".join([rev_word_dict.get(i, "") for i in lst])

In [6]:
training_data = pad_sequences(features, value=0, maxlen=8, padding="post")

In [7]:
model = Sequential()
model.add(Embedding(160, 16))
model.add(GlobalAveragePooling1D())
model.add(Dense(16, activation="relu"))
model.add(Dense(12, activation="softmax"))

model.compile(loss="sparse_categorical_crossentropy", optimizer="adam", metrics=['accuracy'])

In [8]:
model.fit(training_data, labels, epochs=100)

Epoch 1/100
188/188 [==============================] - 2s 3ms/step - loss: 2.2893 - accuracy: 0.3002
Epoch 2/100
188/188 [==============================] - 1s 3ms/step - loss: 1.6332 - accuracy: 0.6070
Epoch 3/100
188/188 [==============================] - 1s 3ms/step - loss: 0.9613 - accuracy: 0.8270
Epoch 4/100
188/188 [==============================] - 1s 3ms/step - loss: 0.5715 - accuracy: 0.9687
Epoch 5/100
188/188 [==============================] - 1s 3ms/step - loss: 0.3222 - accuracy: 0.9947
Epoch 6/100
188/188 [==============================] - 1s 3ms/step - loss: 0.1576 - accuracy: 0.9967
Epoch 7/100
188/188 [==============================] - 1s 3ms/step - loss: 0.0750 - accuracy: 0.9967
Epoch 8/100
188/188 [==============================] - 1s 3ms/step - loss: 0.0409 - accuracy: 0.9972
Epoch 9/100
188/188 [==============================] - 1s 3ms/step - loss: 0.0250 - accuracy: 0.9990
Epoch 10/100
188/188 [==============================] - 1s 3ms/step - loss: 0.0167 - accura

188/188 [==============================] - 1s 3ms/step - loss: 1.6253e-06 - accuracy: 1.0000
Epoch 80/100
188/188 [==============================] - 1s 3ms/step - loss: 1.4652e-06 - accuracy: 1.0000
Epoch 81/100
188/188 [==============================] - 1s 3ms/step - loss: 1.3236e-06 - accuracy: 1.0000
Epoch 82/100
188/188 [==============================] - 1s 3ms/step - loss: 1.1917e-06 - accuracy: 1.0000
Epoch 83/100
188/188 [==============================] - 1s 3ms/step - loss: 1.0748e-06 - accuracy: 1.0000
Epoch 84/100
188/188 [==============================] - 1s 3ms/step - loss: 9.7080e-07 - accuracy: 1.0000
Epoch 85/100
188/188 [==============================] - 1s 3ms/step - loss: 8.7505e-07 - accuracy: 1.0000
Epoch 86/100
188/188 [==============================] - 1s 3ms/step - loss: 7.8920e-07 - accuracy: 1.0000
Epoch 87/100
188/188 [==============================] - 1s 3ms/step - loss: 7.1380e-07 - accuracy: 1.0000
Epoch 88/100
188/188 [==============================] - 1s 

In [11]:
def encode_sent(sent):
    l = sent.lower().replace(".", "").split()
    n = [word_dict.get(i, 0) for i in l]
    return n

def predict(line):
    line = line.lower()
    encoded = encode_sent(line)
    encoded = pad_sequences([encoded], value=0, maxlen=8, padding="post")
    pred = model.predict(encoded)
    val = np.argmax(pred[0])

    line = line.replace(",", " ").split()
    check = lambda x:  x.isdigit()
    convt = lambda x: int(x)
    num_lst = list(filter(check, line))
    num_lst = list(map(convt, num_lst))

    if val == 0:
        sm = 0
        for x in num_lst:
            sm += int(x)
        return f"sum of {num_lst[0]} and {num_lst[1]} is {sm}"

    if val == 1:
        if len(num_lst) != 2:
            raise ValueError(f"Required 2 numbers got {len(num_lst)}")
        sb = num_lst[0] - num_lst[1]
        return f"difference between {num_lst[0]} and {num_lst[1]} is {sb}"

    if val == 2:
        if len(num_lst) != 2:
            raise ValueError(f"Required 2 numbers got {len(num_lst)}")
        ml = num_lst[0] * num_lst[1]
        return f"{num_lst[0]} multiplied by {num_lst[1]} is {ml}"

    if val == 3:
        if len(num_lst) != 2:
            raise ValueError(f"Required 2 numbers got {len(num_lst)}")
        dv = num_lst[0] / num_lst[1]
        return f"{num_lst[0]} divided by {num_lst[1]} is {dv}"

    if val == 4:
        if len(num_lst) != 1:
            raise ValueError(f"Required 1 number got {len(num_lst)}")
        sq = num_lst[0] ** 2
        return f"square of {num_lst[0]} is {sq}"


    if val == 5:
        if len(num_lst) != 1:
            raise ValueError(f"Required 1 number got {len(num_lst)}")
        cb = num_lst[0] ** 3
        return f"cube of {num_lst[0]} is {cb}"

    if val == 6:
        if len(num_lst) != 1:
            raise ValueError(f"Required 1 number got {len(num_lst)}")
        lg = log10(num_lst[0])
        return f"log of {num_lst[0]} is {lg}"

    if val == 7:
        if len(num_lst) != 1:
            raise ValueError(f"Required 1 number got {len(num_lst)}")
        si = sin(radians(num_lst[0]))
        return f"sine value of {num_lst[0]} is {si}"

    if val == 8:
        if len(num_lst) != 1:
            raise ValueError(f"Required 1 number got {len(num_lst)}")
        co = cos(radians(num_lst[0]))
        return f"cos value of {num_lst[0]} is {cs}"

    if val == 9:
        if len(num_lst) != 1:
            raise ValueError(f"Required 1 number got {len(num_lst)}")
        tn = tan(radians(num_lst[0]))
        return f"tan of {num_lst[0]} is {tn}"

    if val == 10:
        if len(num_lst) != 2:
            raise ValueError(f"Required 2 numbers got {len(num_lst)}")
        pw = num_lst[0] ** num_lst[1]
        return f"{num_lst[0]} raised to the power of {num_lst[1]} is {pw}"

    if val == 11:
        if len(num_lst) != 1:
            raise ValueError(f"Required 1 number got {len(num_lst)}")
        fc = 1
        for x in range(1, num_lst[0]+1):
            fc *= x
        return f"Factorial of {num_lst[0]} is {fc}"


In [8]:
import speech_recognition as sr 
r = sr.Recognizer() 
import pyttsx3
engine = pyttsx3.init() 

with sr.Microphone() as source2: 
    r.adjust_for_ambient_noise(source2, duration=0.2) 
    audio2 = r.listen(source2) 
    text = r.recognize_google(audio2) 
    text = text.lower() 
    print(text)
    ret = predict(text) 
    engine.say(ret) 
    engine.runAndWait() 

1
2
what is 7 + 8
